In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
#spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder \
    .master("local") \
    .getOrCreate()

In [2]:
static = spark.read.json("")
dataSchema = static.schema

In [3]:
static.printSchema()
from time import sleep

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [15]:
static.show(5)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
|1424686735500|1424686733498505625|nexus4_1|   99|nexus4|   g|stand|   0.0078125|-0.017654419| 0.010025024|
|1424686735691|1424688581745026978|nexus4_2|  145|nexus4|   g|stand|-3.814697E-4|   0.0184021|-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|  185|nexus4|   g|stand|-3.814697E-4|-0.031799316| -0.00831604|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 5 rows



In [4]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
    .json("")

In [5]:
activityCounts = streaming.groupBy("gt").count()

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [7]:
activityQuery = activityCounts.writeStream.queryName("activity_count")\
    .format("memory").outputMode("complete")\
    .start()

In [ ]:
activityQuery.awaitTermination()

In [8]:
spark.streams.active

In [12]:
from time import sleep
for x in range(5):
    spark.sql("select * from activity_count").show()
    sleep(1)

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       sit| 984714|
|     stand| 910783|
|stairsdown| 749059|
|      walk|1060402|
|  stairsup| 836598|
|      null| 835725|
|      bike| 863710|
+----------+-------+

+----------+-------+
|        gt|  count|
+----------+-------+
|       s

In [9]:
from pyspark.sql.functions import expr
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
    .where("stairs")\
    .where("gt is not null")\
    .select("gt", "model", "arrival_time", "creation_time")\
    .writeStream\
    .queryName("simple_transform")\
    .format("memory")\
    .outputMode("append")\
    .start()

In [10]:
for x in range(5):
    spark.sql("select * from simple_transform").show()
    sleep(1)

+---+-----+------------+-------------+
| gt|model|arrival_time|creation_time|
+---+-----+------------+-------------+
+---+-----+------------+-------------+

+--------+------+-------------+-------------------+
|      gt| model| arrival_time|      creation_time|
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983725|1424687981731838118|
|stairsup|nexus4|1424687984021|1424687982029994857|
|stairsup|nexus4|1424687984421|1424687982426655964|
|stairsup|nexus4|1424687984825|1424687982830373005|
|stairsup|nexus4|1424687985223|1424687983232454474|
|stairsup|nexus4|1424687985633|1424687983635439093|
|stairsup|nexus4|1424687986031|1424687984038271124|
|stairsup|nexus4|1424687986438|1424687984447786505|
|stairsup|nexus4|1424687986837|1424687984843868081|
|stairsup|nexus4|1424687987240|1424687985241725747|
|stairsup|nexus4|1424687987640|1424687985649636253|
|stairsup|nexus4|1424687987997|1424687986007149681|
|stairsup|nexus4|1424687988202|1424689834242463027|
|stairsup|n